<a href="https://colab.research.google.com/github/StokedDude/CANSLIM-signal-booster/blob/main/CANSLIM_Booster_V7_Standardized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 CANSLIM Signal Booster V7 + Personal CIO System

**Complete Trading Intelligence Platform with:**
- ✅ Multi-source ingestion (IBD, DeepVue, Primus, Finviz, X, watchlist)
- ✅ Convergence scoring (multi-source confirmation)
- ✅ Dynamic trust tiers (A/B/C/D auto-classification)
- ✅ Technical overlays (RS, volume, pivot distance, ATR)
- ✅ Instant ticker lookup
- ✅ Daily Brief generator (pre-market intelligence)
- ✅ Portfolio health tracking
- ✅ Market regime detection

---

## 📋 Instructions

1. **Run Cell 1** - Install dependencies
2. **Run Cell 2** - Load all system code
3. **Configure settings** in Cell 3
4. **Run any feature** you need:
   - Cell 4: Upload IBD Data
   - Cell 5: Add External Sources
   - Cell 6: Process & Score Universe
   - Cell 7: Ticker Lookup
   - Cell 8: Generate Daily Brief
   - Cell 9: Export Data

---

In [1]:
#@title 📦 **Cell 1: Install Dependencies** (Run First!)
#@markdown Click the play button to install all required packages.

!pip install -q yfinance pandas numpy scipy scikit-learn tabulate

print("✅ All packages installed!")
print("")
print("Installed:")
print("  • yfinance - Market data")
print("  • pandas/numpy - Data analysis")
print("  • scipy - Statistical functions")
print("  • scikit-learn - ML scoring")
print("  • tabulate - Table formatting")
print("")
print("👉 Now run Cell 2 to load the system code.")

✅ All packages installed!

Installed:
  • yfinance - Market data
  • pandas/numpy - Data analysis
  • scipy - Statistical functions
  • scikit-learn - ML scoring
  • tabulate - Table formatting

👉 Now run Cell 2 to load the system code.


In [2]:
#@title 🧠 **Cell 2: Load System Code** (Run Second!)
#@markdown This cell contains all the system logic. Just run it once.

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Any
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import io

warnings.filterwarnings('ignore')

# ============================================================
# TECHNICAL FACTOR ENGINE
# ============================================================

class TechnicalFactorEngine:
    """Calculate CANSLIM technical factors with caching"""

    def __init__(self):
        self.market_data = {}

    def get_price_data(self, ticker: str, period: str = "6mo") -> pd.DataFrame:
        """Fetch price data with caching"""
        cache_key = f"{ticker}_{period}"
        if cache_key not in self.market_data:
            try:
                data = yf.download(ticker, period=period, progress=False)
                if not data.empty:
                    self.market_data[cache_key] = data
            except:
                return pd.DataFrame()
        return self.market_data.get(cache_key, pd.DataFrame())

    def calculate_rs(self, ticker: str, weeks: int = 13) -> float:
        """Calculate Relative Strength vs SPY"""
        try:
            stock_data = self.get_price_data(ticker)
            spy_data = self.get_price_data("SPY")

            if stock_data.empty or spy_data.empty:
                return 50

            days = min(weeks * 5, len(stock_data) - 1, len(spy_data) - 1)
            if days < 5:
                return 50

            stock_return = (stock_data['Close'].iloc[-1] / stock_data['Close'].iloc[-days] - 1) * 100
            spy_return = (spy_data['Close'].iloc[-1] / spy_data['Close'].iloc[-days] - 1) * 100

            relative_perf = stock_return - spy_return
            rs_score = min(99, max(1, 50 + (relative_perf * 2)))

            return round(rs_score)
        except:
            return 50

    def calculate_volume_trend(self, ticker: str) -> str:
        """Analyze volume pattern"""
        try:
            data = self.get_price_data(ticker)
            if data.empty or 'Volume' not in data.columns or len(data) < 25:
                return "Unknown"

            recent_vol = data['Volume'].iloc[-5:].mean()
            avg_vol = data['Volume'].iloc[-25:-5].mean()

            if avg_vol == 0:
                return "Unknown"

            if recent_vol > avg_vol * 1.2:
                return "Expanding"
            elif recent_vol < avg_vol * 0.8:
                return "Contracting"
            else:
                return "Neutral"
        except:
            return "Unknown"

    def calculate_pivot_distance(self, ticker: str) -> Tuple[float, str]:
        """Calculate distance from 52-week high"""
        try:
            data = self.get_price_data(ticker, period="1y")
            if data.empty:
                return 0, "Unknown"

            current_price = data['Close'].iloc[-1]
            high_52w = data['High'].max()

            if high_52w == 0:
                return 0, "Unknown"

            distance_pct = ((current_price / high_52w) - 1) * 100

            if distance_pct >= -5:
                status = "PROPER SETUP"
            elif distance_pct >= -10:
                status = "NEAR PIVOT"
            elif distance_pct >= -15:
                status = "PULLBACK"
            else:
                status = "EXTENDED DOWN"

            return round(distance_pct, 1), status
        except:
            return 0, "Unknown"

    def calculate_atr(self, ticker: str, period: int = 14) -> float:
        """Calculate Average True Range"""
        try:
            data = self.get_price_data(ticker)
            if data.empty or len(data) < period + 1:
                return 0

            high_low = data['High'] - data['Low']
            high_close = abs(data['High'] - data['Close'].shift())
            low_close = abs(data['Low'] - data['Close'].shift())

            true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
            atr = true_range.rolling(window=period).mean().iloc[-1]

            return round(atr, 2)
        except:
            return 0

    def determine_timeframe(self, atr: float, pivot_distance: float) -> str:
        """Determine trading timeframe"""
        if atr > 5 and abs(pivot_distance) < 5:
            return "DAYTRADE"
        elif abs(pivot_distance) < 5:
            return "SWING"
        elif abs(pivot_distance) < 10:
            return "POSITION"
        else:
            return "WATCH"


# ============================================================
# DATA STORE
# ============================================================

class DataStore:
    """Central data storage for the system"""

    def __init__(self):
        self.ibd_universe = pd.DataFrame()
        self.all_sources_data = defaultdict(list)
        self.ticker_lookup_db = {}
        self.convergence_df = pd.DataFrame()
        self.tech_engine = TechnicalFactorEngine()

    def clear(self):
        """Reset all data"""
        self.ibd_universe = pd.DataFrame()
        self.all_sources_data = defaultdict(list)
        self.ticker_lookup_db = {}
        self.convergence_df = pd.DataFrame()


# ============================================================
# IBD DATA LOADER
# ============================================================

def load_ibd_files(uploaded_files: Dict, data_store: DataStore) -> pd.DataFrame:
    """Load and merge IBD CSV exports"""
    ibd_dataframes = []

    for filename, content in uploaded_files.items():
        try:
            df = pd.read_csv(io.BytesIO(content))
            df.columns = df.columns.str.strip().str.replace(' ', '_')

            ticker_col = None
            for col in ['Symbol', 'Ticker', 'Stock', 'SYMBOL', 'TICKER']:
                if col in df.columns:
                    ticker_col = col
                    break

            if ticker_col:
                df = df.rename(columns={ticker_col: 'Ticker'})
                df['Source_File'] = filename
                ibd_dataframes.append(df)
                print(f"✅ Loaded {filename}: {len(df)} stocks")
            else:
                print(f"⚠️  Skipped {filename}: No ticker column found")

        except Exception as e:
            print(f"❌ Error loading {filename}: {e}")

    if ibd_dataframes:
        data_store.ibd_universe = pd.concat(ibd_dataframes, ignore_index=True)
        data_store.ibd_universe = data_store.ibd_universe.drop_duplicates(subset='Ticker', keep='first')
        data_store.ibd_universe['Ticker'] = data_store.ibd_universe['Ticker'].str.strip().str.upper()

        print(f"\n{'━'*40}")
        print(f"✅ IBD UNIVERSE LOADED")
        print(f"{'━'*40}")
        print(f"Total unique stocks: {len(data_store.ibd_universe)}")
        print(f"Files processed: {len(ibd_dataframes)}")
        print(f"\nSample: {', '.join(data_store.ibd_universe['Ticker'].head(10).tolist())}")

    return data_store.ibd_universe


# ============================================================
# EXTERNAL SOURCE HANDLER
# ============================================================

def add_source_tickers(source_name: str, tickers_input: str, data_store: DataStore) -> List[str]:
    """Add tickers from an external source"""
    tickers = [t.strip().upper() for t in tickers_input.replace(',', ' ').split() if t.strip()]

    for ticker in tickers:
        if source_name not in data_store.all_sources_data[ticker]:
            data_store.all_sources_data[ticker].append(source_name)

    return tickers


# ============================================================
# TECHNICAL OVERLAY PROCESSOR
# ============================================================

def process_technical_overlay(data_store: DataStore, max_workers: int = 12) -> pd.DataFrame:
    """Add technical factors to IBD universe"""

    if data_store.ibd_universe.empty:
        print("❌ No IBD universe to process")
        return pd.DataFrame()

    print("🔄 Processing technical factors...")
    print(f"Analyzing {len(data_store.ibd_universe)} stocks\n")

    # Warm up cache
    _ = data_store.tech_engine.get_price_data("SPY")

    tickers = data_store.ibd_universe['Ticker'].tolist()
    total = len(tickers)

    def compute_one(ticker: str) -> Dict[str, Any]:
        try:
            rs = data_store.tech_engine.calculate_rs(ticker)
            volume = data_store.tech_engine.calculate_volume_trend(ticker)
            pivot_dist, pivot_status = data_store.tech_engine.calculate_pivot_distance(ticker)
            atr = data_store.tech_engine.calculate_atr(ticker)
            timeframe = data_store.tech_engine.determine_timeframe(atr, pivot_dist)

            return {
                'Ticker': ticker,
                'RS_Calculated': rs,
                'Volume_Trend': volume,
                'Pivot_Distance_%': pivot_dist,
                'Pivot_Status': pivot_status,
                'ATR': atr,
                'Timeframe': timeframe,
                'Tech_Status': 'OK'
            }
        except Exception as e:
            return {
                'Ticker': ticker,
                'RS_Calculated': None,
                'Volume_Trend': None,
                'Pivot_Distance_%': None,
                'Pivot_Status': None,
                'ATR': None,
                'Timeframe': None,
                'Tech_Status': f'FAIL: {e}'
            }

    results = []
    chunk_size = 50

    for start in range(0, total, chunk_size):
        chunk = tickers[start:start + chunk_size]

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(compute_one, t): t for t in chunk}
            for fut in as_completed(futures):
                results.append(fut.result())

        processed = min(start + chunk_size, total)
        print(f"Progress: {processed}/{total} ({(processed/total)*100:.0f}%)")

    tech_df = pd.DataFrame(results)
    data_store.ibd_universe = data_store.ibd_universe.merge(tech_df, on='Ticker', how='left')

    print(f"\n✅ Technical overlay complete!")
    return data_store.ibd_universe


# ============================================================
# TIER CLASSIFICATION
# ============================================================

def classify_tier(row: pd.Series) -> str:
    """Assign A/B/C/D tier based on CANSLIM criteria"""

    rs = None
    for col in ['RS_Rating', 'RS', 'Relative_Strength', 'RS_Calculated']:
        if col in row.index and pd.notna(row[col]):
            rs = row[col]
            break
    rs = rs if rs is not None else 50

    composite = None
    for col in ['Composite', 'Composite_Rating', 'Comp', 'Comp_Rating']:
        if col in row.index and pd.notna(row[col]):
            composite = row[col]
            break
    composite = composite if composite is not None else 50

    pivot_status = row.get('Pivot_Status', 'Unknown')

    if rs >= 90 and composite >= 90 and pivot_status == "PROPER SETUP":
        return "A-TIER"
    elif rs >= 80 and composite >= 80:
        return "B-TIER"
    elif rs >= 70 or composite >= 70:
        return "C-TIER"
    else:
        return "D-TIER"


def apply_tier_classification(data_store: DataStore) -> pd.DataFrame:
    """Apply tier classification to universe"""

    if data_store.ibd_universe.empty:
        print("❌ No data to classify")
        return pd.DataFrame()

    data_store.ibd_universe['Tier'] = data_store.ibd_universe.apply(classify_tier, axis=1)
    data_store.ibd_universe['Verification'] = 'IBD-VERIFIED'

    # Build lookup database
    for _, row in data_store.ibd_universe.iterrows():
        data_store.ticker_lookup_db[row['Ticker']] = row.to_dict()

    # Add IBD tickers to sources
    for _, row in data_store.ibd_universe.iterrows():
        ticker = row['Ticker']
        rs = row.get('RS_Rating', row.get('RS_Calculated', 50))
        source = 'IBD_50' if rs and rs >= 90 else 'IBD_250'
        if source not in data_store.all_sources_data[ticker]:
            data_store.all_sources_data[ticker].append(source)

    tier_counts = data_store.ibd_universe['Tier'].value_counts()

    print(f"{'━'*40}")
    print("📊 TIER CLASSIFICATION")
    print(f"{'━'*40}")
    for tier in ['A-TIER', 'B-TIER', 'C-TIER', 'D-TIER']:
        print(f"{tier}: {tier_counts.get(tier, 0)} stocks")
    print(f"\n✅ Classification complete!")
    print(f"✅ Lookup database: {len(data_store.ticker_lookup_db)} stocks")

    return data_store.ibd_universe


# ============================================================
# CONVERGENCE SCORING
# ============================================================

SOURCE_WEIGHTS = {
    'IBD_50': 3,
    'IBD_250': 2,
    'DeepVue': 2,
    'Primus': 2,
    'Finviz': 1,
    'X_Mentions': 1,
    'Watchlist': 1
}


def calculate_convergence(data_store: DataStore) -> pd.DataFrame:
    """Calculate convergence scores for all tickers"""

    convergence_data = []
    max_weight = sum(SOURCE_WEIGHTS.values())

    for ticker, sources in data_store.all_sources_data.items():
        total_weight = sum(SOURCE_WEIGHTS.get(s, 1) for s in sources)
        score = min(10, int((total_weight / max_weight) * 10))

        if score >= 8:
            stars = "⭐⭐⭐"
        elif score >= 6:
            stars = "⭐⭐"
        elif score >= 4:
            stars = "⭐"
        else:
            stars = ""

        convergence_data.append({
            'Ticker': ticker,
            'Sources': ', '.join(sources),
            'Source_Count': len(sources),
            'Convergence_Score': score,
            'Stars': stars
        })

    data_store.convergence_df = pd.DataFrame(convergence_data).sort_values(
        'Convergence_Score', ascending=False
    )

    high = len(data_store.convergence_df[data_store.convergence_df['Convergence_Score'] >= 8])
    med = len(data_store.convergence_df[data_store.convergence_df['Convergence_Score'].between(6, 7)])

    print(f"{'━'*40}")
    print("⭐ CONVERGENCE SCORING")
    print(f"{'━'*40}")
    print(f"Total tickers: {len(data_store.convergence_df)}")
    print(f"High conviction (⭐⭐⭐): {high}")
    print(f"Medium conviction (⭐⭐): {med}")
    print(f"\n✅ Convergence analysis complete!")

    return data_store.convergence_df


# ============================================================
# TICKER LOOKUP
# ============================================================

def lookup_ticker(ticker: str, data_store: DataStore) -> None:
    """Instant ticker quality lookup"""

    ticker = ticker.strip().upper()

    print(f"\n{'='*50}")
    print(f"📊 TICKER LOOKUP: {ticker}")
    print(f"{'='*50}")

    conv_row = data_store.convergence_df[
        data_store.convergence_df['Ticker'] == ticker
    ] if not data_store.convergence_df.empty else pd.DataFrame()

    if ticker in data_store.ticker_lookup_db:
        data = data_store.ticker_lookup_db[ticker]

        sources = conv_row['Sources'].iloc[0] if not conv_row.empty else 'IBD only'
        stars = conv_row['Stars'].iloc[0] if not conv_row.empty else ''

        tier = data.get('Tier', 'UNKNOWN')
        print(f"\n✅ {tier} - IBD VERIFIED")
        print(f"\nConviction: {stars}")
        print(f"Sources: {sources}")

        print("\n📈 CANSLIM Ratings:")
        for col in ['Composite', 'Comp_Rating', 'RS_Rating', 'EPS_Rating', 'SMR_Rating', 'A/D_Rating']:
            if col in data and pd.notna(data[col]):
                print(f"  {col}: {data[col]}")

        print("\n📊 Technical Analysis:")
        print(f"  Pivot Status: {data.get('Pivot_Status', 'N/A')}")
        print(f"  Distance from Pivot: {data.get('Pivot_Distance_%', 'N/A')}%")
        print(f"  Volume Trend: {data.get('Volume_Trend', 'N/A')}")
        print(f"  ATR: ${data.get('ATR', 'N/A')}")
        print(f"  Timeframe: {data.get('Timeframe', 'N/A')}")

        pivot_status = str(data.get('Pivot_Status', '')).upper()

        print("\n💡 Recommendation:")
        if tier == 'A-TIER' and pivot_status == 'PROPER SETUP':
            print("  🟢 HIGH CONVICTION - Ready to trade")
        elif tier == 'A-TIER':
            print("  🟡 QUALITY STOCK - Wait for better entry")
        elif tier == 'B-TIER' and pivot_status == 'PROPER SETUP':
            print("  🟡 WATCHABLE - Consider smaller position")
        elif tier == 'B-TIER':
            print("  🟡 WATCH - Set alerts for entry")
        else:
            print("  🔴 PASS - Low conviction")

    else:
        print("\n⚠️  NOT IN IBD DATABASE")

        if not conv_row.empty:
            sources = conv_row['Sources'].iloc[0]
            print(f"\nFound in: {sources}")
            print("\nCalculating technical scores...")

            try:
                rs = data_store.tech_engine.calculate_rs(ticker)
                volume = data_store.tech_engine.calculate_volume_trend(ticker)
                pivot_dist, pivot_status = data_store.tech_engine.calculate_pivot_distance(ticker)
                atr = data_store.tech_engine.calculate_atr(ticker)

                print("\n📊 TECHNICAL-ONLY Analysis:")
                print(f"  RS (estimated): {rs}")
                print(f"  Volume: {volume}")
                print(f"  Pivot Status: {pivot_status} ({pivot_dist}%)")
                print(f"  ATR: ${atr}")

                if rs >= 80 and pivot_status == 'PROPER SETUP':
                    grade = "B-TIER (Technical-only)"
                    rec = "🟡 WATCHABLE - No fundamental verification"
                elif rs >= 70:
                    grade = "C-TIER (Technical-only)"
                    rec = "🟠 DAYTRADE ONLY - Tight stops"
                else:
                    grade = "D-TIER"
                    rec = "🔴 PASS - Weak metrics"

                print(f"\nGrade: {grade}")
                print(f"\n💡 Recommendation: {rec}")

            except Exception as e:
                print(f"\n❌ Technical calc failed: {e}")
        else:
            print("\n❌ Not found in any source")

    print(f"\n{'='*50}")


# ============================================================
# DAILY BRIEF GENERATOR
# ============================================================

def generate_daily_brief(data_store: DataStore, top_daytrades: int = 5, top_swings: int = 3) -> None:
    """Generate comprehensive pre-market daily brief"""

    current_date = datetime.now().strftime('%A, %B %d, %Y')

    print(f"\n{'='*60}")
    print("📋 DAILY TRADING BRIEF")
    print(f"{current_date} | Pre-Market")
    print(f"{'='*60}")

    # Market Regime
    try:
        spy_data = data_store.tech_engine.get_price_data('SPY', period='1mo')
        if len(spy_data) >= 6:
            spy_return = ((spy_data['Close'].iloc[-1] / spy_data['Close'].iloc[-5]) - 1) * 100

            vix_data = data_store.tech_engine.get_price_data('^VIX', period='1mo')
            vix = float(vix_data['Close'].iloc[-1]) if not vix_data.empty else 15.0

            if spy_return > 1 and vix < 15:
                regime = "Trending Up ✅"
                sizing = "Full position sizing OK"
            elif spy_return < -1 or vix > 20:
                regime = "Choppy / Distribution ⚠️"
                sizing = "Reduce size 50%"
            else:
                regime = "Neutral"
                sizing = "Normal sizing"

            print(f"\nMARKET REGIME: {regime}")
            print(f"Recommendation: {sizing}")
            print(f"SPY (5-day): {spy_return:+.1f}% | VIX: {vix:.1f}")
    except Exception as e:
        print(f"\nMARKET REGIME: Unable to fetch ({e})")

    print(f"\n{'━'*60}")

    # Merge convergence data
    if not data_store.ibd_universe.empty and not data_store.convergence_df.empty:
        merged = data_store.ibd_universe.merge(
            data_store.convergence_df[['Ticker', 'Stars', 'Sources', 'Convergence_Score']],
            on='Ticker', how='left'
        )
    else:
        merged = data_store.ibd_universe.copy()

    # Top Daytrades
    print(f"\n🎯 TOP {top_daytrades} DAYTRADE SETUPS\n")

    if 'Timeframe' in merged.columns and 'Tier' in merged.columns:
        daytrades = merged[
            (merged['Timeframe'] == 'DAYTRADE') &
            (merged['Tier'].isin(['A-TIER', 'B-TIER']))
        ].head(top_daytrades)

        if daytrades.empty:
            print("   No daytrade setups matching criteria")
        else:
            for i, row in daytrades.iterrows():
                stars = row.get('Stars', '')
                print(f"{row['Ticker']} - {row['Tier']} {stars}")
                print(f"   Setup: {row.get('Pivot_Status', 'N/A')} | ATR: ${row.get('ATR', 'N/A')}")
    else:
        print("   No data available")

    print(f"\n{'━'*60}")

    # Top Swings
    print(f"\n📈 TOP {top_swings} SWING SETUPS\n")

    if 'Timeframe' in merged.columns and 'Tier' in merged.columns:
        swings = merged[
            (merged['Timeframe'].isin(['SWING', 'POSITION'])) &
            (merged['Tier'].isin(['A-TIER', 'B-TIER']))
        ].head(top_swings)

        if swings.empty:
            print("   No swing setups matching criteria")
        else:
            for i, row in swings.iterrows():
                stars = row.get('Stars', '')
                print(f"{row['Ticker']} - {row['Tier']} {stars}")
                print(f"   Setup: {row.get('Pivot_Status', 'N/A')}")
    else:
        print("   No data available")

    print(f"\n{'━'*60}")

    # Watch List
    print("\n👁️  WATCH LIST\n")

    if 'Timeframe' in merged.columns:
        watch = merged[
            (merged['Timeframe'] == 'WATCH') &
            (merged['Tier'].isin(['A-TIER', 'B-TIER']))
        ]
        if not watch.empty:
            tickers = watch['Ticker'].head(10).tolist()
            print(f"   {', '.join(tickers)}")
            if len(watch) > 10:
                print(f"   ...and {len(watch) - 10} more")
        else:
            print("   No stocks on watch")

    print(f"\n{'='*60}")
    print("✅ Daily Brief Complete")
    print(f"{'='*60}")


# ============================================================
# EXPORT FUNCTIONS
# ============================================================

def export_to_csv(data_store: DataStore, filename: str = None) -> str:
    """Export daily brief to CSV"""

    if data_store.ibd_universe.empty:
        print("❌ No data to export")
        return None

    if filename is None:
        filename = f"canslim_brief_{datetime.now().strftime('%Y%m%d')}.csv"

    # Merge with convergence
    if not data_store.convergence_df.empty:
        export_df = data_store.ibd_universe.merge(
            data_store.convergence_df[['Ticker', 'Stars', 'Sources', 'Convergence_Score']],
            on='Ticker', how='left'
        )
    else:
        export_df = data_store.ibd_universe.copy()

    export_df = export_df.sort_values(['Tier', 'Convergence_Score'] if 'Convergence_Score' in export_df.columns else ['Tier'],
                                       ascending=[True, False] if 'Convergence_Score' in export_df.columns else [True])

    export_df.to_csv(filename, index=False)

    tier_counts = export_df['Tier'].value_counts() if 'Tier' in export_df.columns else {}

    print(f"✅ Exported to {filename}")
    print(f"   Total: {len(export_df)} stocks")
    print(f"   A-TIER: {tier_counts.get('A-TIER', 0)}")
    print(f"   B-TIER: {tier_counts.get('B-TIER', 0)}")

    return filename


# ============================================================
# INITIALIZE SYSTEM
# ============================================================

# Create global data store
DATA = DataStore()

print("✅ All system code loaded!")
print("")
print("Components ready:")
print("  • TechnicalFactorEngine - RS, volume, ATR calculations")
print("  • DataStore - Central data management")
print("  • Tier Classification - A/B/C/D ranking")
print("  • Convergence Scoring - Multi-source confirmation")
print("  • Daily Brief Generator - Pre-market intelligence")
print("")
print("👉 Now configure your settings in Cell 3.")

✅ All system code loaded!

Components ready:
  • TechnicalFactorEngine - RS, volume, ATR calculations
  • DataStore - Central data management
  • Tier Classification - A/B/C/D ranking
  • Convergence Scoring - Multi-source confirmation
  • Daily Brief Generator - Pre-market intelligence

👉 Now configure your settings in Cell 3.


In [10]:
#@title ⚙️ **Cell 3: Configuration**
#@markdown ### Processing Settings
MAX_WORKERS = 12 #@param {type:"slider", min:4, max:24, step:4}

#@markdown ### Daily Brief Settings
TOP_DAYTRADES = 5 #@param {type:"slider", min:3, max:10, step:1}
TOP_SWINGS = 5 #@param {type:"slider", min:2, max:8, step:1}

#@markdown ### Tier Thresholds
A_TIER_RS_MIN = 90 #@param {type:"slider", min:80, max:99, step:5}
B_TIER_RS_MIN = 80 #@param {type:"slider", min:70, max:90, step:5}
C_TIER_RS_MIN = 70 #@param {type:"slider", min:60, max:80, step:5}

#@markdown ### Source Weights (for convergence scoring)
WEIGHT_IBD_50 = 3 #@param {type:"slider", min:1, max:5, step:1}
WEIGHT_IBD_250 = 2 #@param {type:"slider", min:1, max:5, step:1}
WEIGHT_DEEPVUE = 2 #@param {type:"slider", min:1, max:5, step:1}
WEIGHT_PRIMUS = 2 #@param {type:"slider", min:1, max:5, step:1}
WEIGHT_FINVIZ = 2 #@param {type:"slider", min:1, max:5, step:1}
WEIGHT_X = 2 #@param {type:"slider", min:1, max:5, step:1}
WEIGHT_WATCHLIST = 5 #@param {type:"slider", min:1, max:5, step:1}

# Update source weights
SOURCE_WEIGHTS.update({
    'IBD_50': WEIGHT_IBD_50,
    'IBD_250': WEIGHT_IBD_250,
    'DeepVue': WEIGHT_DEEPVUE,
    'Primus': WEIGHT_PRIMUS,
    'Finviz': WEIGHT_FINVIZ,
    'X_Mentions': WEIGHT_X,
    'Watchlist': WEIGHT_WATCHLIST
})

# Store config
CONFIG = {
    'max_workers': MAX_WORKERS,
    'top_daytrades': TOP_DAYTRADES,
    'top_swings': TOP_SWINGS,
    'a_tier_rs': A_TIER_RS_MIN,
    'b_tier_rs': B_TIER_RS_MIN,
    'c_tier_rs': C_TIER_RS_MIN,
}

print("✅ Configuration saved!")
print(f"""
Settings:
  Max Workers: {MAX_WORKERS}
  Top Daytrades: {TOP_DAYTRADES}
  Top Swings: {TOP_SWINGS}

Tier Thresholds:
  A-TIER: RS >= {A_TIER_RS_MIN}
  B-TIER: RS >= {B_TIER_RS_MIN}
  C-TIER: RS >= {C_TIER_RS_MIN}

Source Weights:
  IBD 50: {WEIGHT_IBD_50} | IBD 250: {WEIGHT_IBD_250}
  DeepVue: {WEIGHT_DEEPVUE} | Primus: {WEIGHT_PRIMUS}
  Finviz: {WEIGHT_FINVIZ} | X: {WEIGHT_X} | Watchlist: {WEIGHT_WATCHLIST}
""")
print("👉 Now upload your IBD data in Cell 4.")

✅ Configuration saved!

Settings:
  Max Workers: 12
  Top Daytrades: 5
  Top Swings: 5
  
Tier Thresholds:
  A-TIER: RS >= 90
  B-TIER: RS >= 80
  C-TIER: RS >= 70
  
Source Weights:
  IBD 50: 3 | IBD 250: 2
  DeepVue: 2 | Primus: 2
  Finviz: 2 | X: 2 | Watchlist: 5

👉 Now upload your IBD data in Cell 4.


In [11]:
#@title 📂 **Cell 4: Upload IBD Data**
#@markdown Upload your IBD CSV exports (IBD 50, IBD 250, Sector Leaders, etc.)

from google.colab import files

print("📤 Upload your IBD CSV files")
print("You can select multiple files at once...\n")

uploaded = files.upload()

if uploaded:
    load_ibd_files(uploaded, DATA)
else:
    print("❌ No files uploaded")

📤 Upload your IBD CSV files
You can select multiple files at once...



Saving Deepvue Leaders.csv to Deepvue Leaders (1).csv
Saving IBD 50 Index.csv to IBD 50 Index (1).csv
Saving IBD Big Cap 20.csv to IBD Big Cap 20 (1).csv
Saving MarketSurge Growth 250.csv to MarketSurge Growth 250 (1).csv
✅ Loaded Deepvue Leaders (1).csv: 291 stocks
✅ Loaded IBD 50 Index (1).csv: 50 stocks
✅ Loaded IBD Big Cap 20 (1).csv: 20 stocks
✅ Loaded MarketSurge Growth 250 (1).csv: 299 stocks

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ IBD UNIVERSE LOADED
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Total unique stocks: 498
Files processed: 4

Sample: CALY, SNDK, EQPT, IBRX, ENPH, SLAB, AAP, AMN, VSNT, BE


In [12]:
#@title 📥 **Cell 5: Add External Sources**
#@markdown Enter tickers from other sources (comma or space separated)

DEEPVUE_TICKERS = "" #@param {type:"string"}
PRIMUS_TICKERS = "" #@param {type:"string"}
FINVIZ_TICKERS = "" #@param {type:"string"}
X_MENTIONS = "" #@param {type:"string"}
WATCHLIST = "" #@param {type:"string"}

print("━" * 40)
print("📥 ADDING EXTERNAL SOURCES")
print("━" * 40 + "\n")

sources_added = 0

if DEEPVUE_TICKERS.strip():
    tickers = add_source_tickers('DeepVue', DEEPVUE_TICKERS, DATA)
    print(f"✅ DeepVue: {len(tickers)} tickers added")
    sources_added += len(tickers)

if PRIMUS_TICKERS.strip():
    tickers = add_source_tickers('Primus', PRIMUS_TICKERS, DATA)
    print(f"✅ Primus: {len(tickers)} tickers added")
    sources_added += len(tickers)

if FINVIZ_TICKERS.strip():
    tickers = add_source_tickers('Finviz', FINVIZ_TICKERS, DATA)
    print(f"✅ Finviz: {len(tickers)} tickers added")
    sources_added += len(tickers)

if X_MENTIONS.strip():
    tickers = add_source_tickers('X_Mentions', X_MENTIONS, DATA)
    print(f"✅ X/Twitter: {len(tickers)} tickers added")
    sources_added += len(tickers)

if WATCHLIST.strip():
    tickers = add_source_tickers('Watchlist', WATCHLIST, DATA)
    print(f"✅ Watchlist: {len(tickers)} tickers added")
    sources_added += len(tickers)

if sources_added == 0:
    print("ℹ️  No external sources added (all fields empty)")
else:
    print(f"\n✅ Total unique tickers from all sources: {len(DATA.all_sources_data)}")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📥 ADDING EXTERNAL SOURCES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ℹ️  No external sources added (all fields empty)


In [15]:
#@title 🔄 **Cell 6: Process & Score Universe**
#@markdown Run technical analysis, tier classification, and convergence scoring.

RUN_TECHNICAL_OVERLAY = True #@param {type:"boolean"}

if 'CONFIG' not in globals():
    print("❌ Error: Configuration (Cell 3) has not been loaded. Please run Cell 3 first.")
    print("Skipping processing until configuration is defined.")
elif DATA.ibd_universe.empty:
    print("❌ No IBD data loaded. Please run Cell 4 first.")
else:
    print("🔄 Processing universe...\n")

    # Step 1: Technical overlay (optional - takes time)
    if RUN_TECHNICAL_OVERLAY:
        process_technical_overlay(DATA, max_workers=CONFIG['max_workers'])
        print("")
    else:
        print("⏭️  Skipping technical overlay")
        print("   (Enable RUN_TECHNICAL_OVERLAY for full analysis)\n")

    # Ensure relevant rating columns are numeric before classification
    rating_cols_to_convert = [
        'RS_Rating', 'RS', 'Relative_Strength',
        'Composite', 'Composite_Rating', 'Comp', 'Comp_Rating'
    ]
    for col in rating_cols_to_convert:
        if col in DATA.ibd_universe.columns:
            DATA.ibd_universe[col] = pd.to_numeric(DATA.ibd_universe[col], errors='coerce')

    # Step 2: Tier classification
    apply_tier_classification(DATA)
    print("")

    # Step 3: Convergence scoring
    calculate_convergence(DATA)

    print("\n" + "━" * 40)
    print("✅ PROCESSING COMPLETE")
    print("━" * 40)
    print(f"Universe: {len(DATA.ibd_universe)} stocks")
    print(f"Lookup DB: {len(DATA.ticker_lookup_db)} entries")
    print(f"\n👉 Use Cell 7 for ticker lookup or Cell 8 for daily brief.")

🔄 Processing universe...

🔄 Processing technical factors...
Analyzing 498 stocks

Progress: 50/498 (10%)
Progress: 100/498 (20%)
Progress: 150/498 (30%)
Progress: 200/498 (40%)
Progress: 250/498 (50%)
Progress: 300/498 (60%)
Progress: 350/498 (70%)
Progress: 400/498 (80%)
Progress: 450/498 (90%)
Progress: 498/498 (100%)

✅ Technical overlay complete!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 TIER CLASSIFICATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
A-TIER: 0 stocks
B-TIER: 160 stocks
C-TIER: 46 stocks
D-TIER: 292 stocks

✅ Classification complete!
✅ Lookup database: 498 stocks

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⭐ CONVERGENCE SCORING
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Total tickers: 498
High conviction (⭐⭐⭐): 0
Medium conviction (⭐⭐): 0

✅ Convergence analysis complete!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ PROCESSING COMPLETE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Universe: 498 stocks
Lookup DB: 498 entries

👉 Use Cell 7 for ticker lookup or Cell 8 for daily b

In [16]:
#@title 🔍 **Cell 7: Ticker Lookup**
#@markdown Enter any ticker for instant quality analysis.

TICKER = "GOOGL" #@param {type:"string"}

if TICKER.strip():
    lookup_ticker(TICKER, DATA)
else:
    print("❌ Please enter a ticker symbol")


📊 TICKER LOOKUP: GOOGL

✅ B-TIER - IBD VERIFIED

Conviction: 
Sources: IBD_50

📈 CANSLIM Ratings:
  Comp_Rating: 98.0
  RS_Rating: 93.0
  EPS_Rating: 92.0
  SMR_Rating: A
  A/D_Rating: A-

📊 Technical Analysis:
  Pivot Status: Unknown
  Distance from Pivot: 0%
  Volume Trend: Unknown
  ATR: $0.66
  Timeframe: SWING

💡 Recommendation:
  🟡 WATCH - Set alerts for entry



In [17]:
#@title 📋 **Cell 8: Generate Daily Brief**
#@markdown Generate comprehensive pre-market intelligence report.

generate_daily_brief(DATA, top_daytrades=CONFIG['top_daytrades'], top_swings=CONFIG['top_swings'])


📋 DAILY TRADING BRIEF
Thursday, February 05, 2026 | Pre-Market

MARKET REGIME: Unable to fetch (The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 TOP 5 DAYTRADE SETUPS

RKLB - B-TIER 
   Setup: Unknown | ATR: $8.21
STRL - B-TIER 
   Setup: Unknown | ATR: $8.21
DY - B-TIER 
   Setup: Unknown | ATR: $8.21
NVMI - B-TIER 
   Setup: Unknown | ATR: $8.21
FIVE - B-TIER 
   Setup: Unknown | ATR: $8.21

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📈 TOP 5 SWING SETUPS

FIX - B-TIER 
   Setup: Unknown
APLD - B-TIER 
   Setup: Unknown
AEM - B-TIER 
   Setup: Unknown
APH - B-TIER 
   Setup: Unknown
AAOI - B-TIER 
   Setup: Unknown

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

👁️  WATCH LIST

   No stocks on watch

✅ Daily Brief Complete


In [18]:
#@title 📊 **Cell 9: Export Data**
#@markdown Export the processed data to CSV.

CUSTOM_FILENAME = "" #@param {type:"string"}
DOWNLOAD_FILE = True #@param {type:"boolean"}

filename = CUSTOM_FILENAME if CUSTOM_FILENAME.strip() else None
exported_file = export_to_csv(DATA, filename)

if exported_file and DOWNLOAD_FILE:
    from google.colab import files
    files.download(exported_file)

✅ Exported to canslim_brief_20260205.csv
   Total: 498 stocks
   A-TIER: 0
   B-TIER: 160


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
#@title 🎮 **Cell 10: Quick Commands**
#@markdown Utility functions for interactive use.

def show_a_tier():
    """Display all A-TIER stocks"""
    if DATA.ibd_universe.empty:
        print("❌ No data loaded")
        return

    a_tier = DATA.ibd_universe[DATA.ibd_universe['Tier'] == 'A-TIER']

    print(f"\n{'━'*40}")
    print(f"⭐ A-TIER STOCKS ({len(a_tier)} total)")
    print(f"{'━'*40}\n")

    if a_tier.empty:
        print("No A-TIER stocks found")
    else:
        for _, row in a_tier.iterrows():
            print(f"{row['Ticker']} - {row.get('Pivot_Status', 'N/A')} | {row.get('Timeframe', 'N/A')}")


def show_high_conviction():
    """Display high conviction (multi-source) stocks"""
    if DATA.convergence_df.empty:
        print("❌ Run Cell 6 first to calculate convergence")
        return

    high = DATA.convergence_df[DATA.convergence_df['Convergence_Score'] >= 8]

    print(f"\n{'━'*40}")
    print(f"⭐⭐⭐ HIGH CONVICTION ({len(high)} stocks)")
    print(f"{'━'*40}\n")

    if high.empty:
        print("No high conviction stocks (need multi-source confirmation)")
    else:
        for _, row in high.iterrows():
            tier = DATA.ticker_lookup_db.get(row['Ticker'], {}).get('Tier', 'Unknown')
            print(f"{row['Ticker']} - {tier} | Sources: {row['Sources']}")


def show_by_timeframe(timeframe: str):
    """Display stocks by timeframe (DAYTRADE, SWING, POSITION, WATCH)"""
    if DATA.ibd_universe.empty or 'Timeframe' not in DATA.ibd_universe.columns:
        print("❌ Run Cell 6 first")
        return

    tf = timeframe.upper()
    stocks = DATA.ibd_universe[DATA.ibd_universe['Timeframe'] == tf]

    print(f"\n{'━'*40}")
    print(f"{tf} CANDIDATES ({len(stocks)} total)")
    print(f"{'━'*40}\n")

    for _, row in stocks.head(20).iterrows():
        print(f"{row['Ticker']} - {row.get('Tier', 'N/A')} | ATR: ${row.get('ATR', 'N/A')}")

    if len(stocks) > 20:
        print(f"\n...and {len(stocks) - 20} more")


print("✅ Quick commands loaded!")
print("""
Available commands:
  • lookup_ticker('SYMBOL', DATA) - Quick ticker analysis
  • show_a_tier() - View A-TIER stocks
  • show_high_conviction() - Multi-source plays
  • show_by_timeframe('DAYTRADE') - Filter by timeframe
  • generate_daily_brief(DATA) - Full pre-market brief
  • export_to_csv(DATA) - Export to file

Examples:
  lookup_ticker('AAPL', DATA)
  show_by_timeframe('SWING')
""")

✅ Quick commands loaded!

Available commands:
  • lookup_ticker('SYMBOL', DATA) - Quick ticker analysis
  • show_a_tier() - View A-TIER stocks
  • show_high_conviction() - Multi-source plays
  • show_by_timeframe('DAYTRADE') - Filter by timeframe
  • generate_daily_brief(DATA) - Full pre-market brief
  • export_to_csv(DATA) - Export to file

Examples:
  lookup_ticker('AAPL', DATA)
  show_by_timeframe('SWING')



---

## 📚 Quick Reference

### Tier Classification

| Tier | Criteria | Action |
|------|----------|--------|
| A-TIER | RS ≥ 90, Composite ≥ 90, Proper Setup | Full position |
| B-TIER | RS ≥ 80, Composite ≥ 80 | Smaller position |
| C-TIER | RS ≥ 70 OR Composite ≥ 70 | Daytrade only |
| D-TIER | Below thresholds | Pass |

### Convergence Scoring

| Stars | Score | Meaning |
|-------|-------|----------|
| ⭐⭐⭐ | 8-10 | High conviction (multiple quality sources) |
| ⭐⭐ | 6-7 | Medium conviction |
| ⭐ | 4-5 | Low conviction |
| - | 0-3 | Single source only |

### Timeframe Classification

| Timeframe | Criteria | Holding Period |
|-----------|----------|----------------|
| DAYTRADE | ATR > $5, within 5% of pivot | Intraday |
| SWING | Within 5% of pivot | 2-5 days |
| POSITION | Within 10% of pivot | 1-4 weeks |
| WATCH | Extended from pivot | Wait for setup |

### Source Weights (Default)

| Source | Weight | Description |
|--------|--------|-------------|
| IBD 50 | 3 | Top institutional quality |
| IBD 250 | 2 | Growth leaders |
| DeepVue | 2 | True market leaders |
| Primus | 2 | Pre-market momentum |
| Finviz | 1 | Technical scans |
| X/Twitter | 1 | Social mentions |
| Watchlist | 1 | Personal picks |

---

## 📈 Daily Workflow

1. **Pre-market (5 min)**
   - Upload fresh IBD exports (Cell 4)
   - Add Primus/DeepVue tickers (Cell 5)
   - Process universe (Cell 6)

2. **Generate Brief (instant)**
   - Run Cell 8 for daily brief
   - Focus on top daytrades + swings

3. **During Market**
   - Use Cell 7 for quick lookups
   - Trust A-TIER + ⭐⭐⭐ convergence

4. **Post-market**
   - Export results (Cell 9)
   - Review for next day

---

*Built with the standardized notebook template | Version 7.0*